In [ ]:
import numpy as np
import pandas as pd
import astropy.constants as const
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Parameters that you can change for new model
c = const.c.cgs.value #speed of light in cm/s
c_factor = 0.71 #factor to change the speed of light for the velocity of forward shock (needs to be above 0.7 for a_gamma_sf to be positive)
v_fs = c_factor * c #velocity of forward shock in cm/s
dens = 1e-15 #density of agn disk

b_fs = v_fs / c
b_sf = 0.5 #seen on page 5
e_e = 0.3 #page 8 top right
e_b = 0.1 #page 8 top right 
xi = 1 # USED TO GET GAMMA MAX NOT SURE WHERE FROM
p = 2.5 #foud on page 8 top right
gamma_fs = 1 / np.sqrt(1 - b_fs**2)
gamma_fs_f = gamma_fs**(1 + np.sqrt(3))
gamma_sf = gamma_fs / np.sqrt(2)
gamma_sf_f = gamma_sf**(1 + np.sqrt(3))
a_gamma_sf = (gamma_sf -1)*(4*gamma_sf+3) #left of eq 24
gamma_m = 40 * (e_e/0.3)*(gamma_sf-1/0.2) #used in eq 25 from eq 16
gamma_max = 1e6 * xi**(-0.5) * (b_sf/0.5)*((a_gamma_sf/1.5)**(-0.25))*((e_b/0.1)**(-0.25))*((dens/1e-16)**(-0.25)) #used in eq 26 from eq 17
gamma_a = 60 #used in eq 27 has a messy equation at (21) pls say we dont need it
n = dens / const.m_p

def lum_sh(): #will get from emily
    if b_fs * gamma_fs <=1:
        lum_sh = 42
    else: #b_fs * gamma_fs > 1
        lum_sh = 44
    return lum_sh

def lum_syn(): #eq 15
    Yssc = (e_e / e_b)**(1/3) #page 8 top left
    Y2ndIC = (e_e / e_b)**(2/3) #page 8 top left
    f_dilution = 1 #page 6 bottom left
    if b_fs * gamma_fs <= 1:
        f_beaming = 1
    else: #b_fs * gamma_fs > 1
        f_beaming = 2 * (gamma_sf_f**2)
    lum_syn = e_e * f_dilution * f_beaming * (lum_sh() / (1 + Yssc + Y2ndIC))
    return lum_syn

freq = (1e17) #frequency range to test in Hz

def lum_tot(freq):
    B_sf = 6e2*((a_gamma_sf/1.5)**0.5) * ((e_b/0.1)**0.5) * (dens/1e-16) #left of eq 24 (NOT NUMBERERED)
    freq_sync = 2e9 * (gamma_sf/1.2) * (B_sf/600) #eq 24
    freq_m = 3e12*((gamma_m/40)**2)*(freq_sync/2e9) #eq 25
    freq_max = 2e21*((gamma_max/1e6)**2)*(freq_sync/2e9) #eq 26 (need to ask if i use the first or second equation)
    freq_a = 7e12 * ((gamma_a/60)**2) * (freq_sync/2e9) #eq 27
    freq_combo = 0
    
    if freq_a < freq_m: #eq 28
        if freq_m < freq < freq_max:
            freq_combo = (freq/freq_m)**(-(p-1)/2)
        elif freq_a < freq < freq_m:
            freq_combo = (freq/freq_m)**(0.5)
        elif freq_sync < freq < freq_a:
            freq_combo = ((freq/freq_a)**(3))*((freq_a/freq_m)**(0.5))
        else:
            freq_combo = 0.1
    if freq_m < freq_a: #eq 29
        if freq_a < freq < freq_max:
            freq_combo = (freq/freq_a)**(-(p-1)/2) + np.exp(1-(freq/freq_a)**(0.5))
        elif freq < freq_a:
            freq_combo = (freq/freq_a)**3
        else:
            freq_combo = 0

    lum_tot = (lum_syn() * freq_combo)/freq
    return lum_tot

test = lum_tot(freq)

print(test, a_gamma_sf)
print("xi =", xi)
print("a_gamma_sf =", a_gamma_sf)
print("e_b =", e_b)
print("dens =", dens)
print("b_sf =", b_sf)

2.1641380468713992e-19 0.02894579515842816
xi = 1
a_gamma_sf = 0.02894579515842816
e_b = 0.1
dens = 1e-15
b_sf = 0.5


In [ ]:
B_sf = 6e2*((a_gamma_sf/1.5)**0.5) * ((e_b/0.1)**0.5) * (dens/1e-16) #left of eq 24 (NOT NUMBERERED)
freq_sync = 2e9 * (gamma_sf/1.2) * (B_sf/600) #eq 24
freq_m = 3e12*((gamma_m/40)**2)*(freq_sync/2e9) #eq 25
freq_max = 2e21*((gamma_max/1e6)**2)*(freq_sync/2e9) #eq 26 (need to ask if i use the first or second equation)
freq_a = 7e12 * ((gamma_a/60)**2) * (freq_sync/2e9) #eq 27
freq_combo = 0
    
if freq_a < freq_m: #eq 28
    if freq_m < freq < freq_max:
        freq_combo = (freq/freq_m)**(-(p-1)/2)
    elif freq_a < freq < freq_m:
        freq_combo = (freq/freq_m)**(0.5)
    elif freq_sync < freq < freq_a:
        freq_combo = ((freq/freq_a)**(3))*((freq_a/freq_m)**(0.5))
    else:
        freq_combo = 0.1
if freq_m < freq_a: #eq 29
    if freq_a < freq < freq_max:
        freq_combo = (freq/freq_a)**(-(p-1)/2) + np.exp(1-(freq/freq_a)**(0.5))
    elif freq < freq_a:
        req_combo = (freq/freq_a)**3
    else:
        freq_combo = 0

    lum_tot = (lum_syn() * freq_combo)/freq

print(lum_tot, freq_a, freq_m, freq_max)
)

<function lum_tot at 0x307630680> nan nan nan
xi = 1
a_gamma_sf = -1.390564368858079
e_b = 0.1
dens = 1e-15
b_sf = 0.5


/var/folders/pw/nc5rj5md667b29kntncq7jbw0000gn/T/ipykernel_45601/4287191176.py:1: RuntimeWarning: invalid value encountered in scalar power
  B_sf = 6e2*((a_gamma_sf/1.5)**0.5) * ((e_b/0.1)**0.5) * (dens/1e-16) #left of eq 24 (NOT NUMBERERED)
